## PlayStore

In [14]:
import os

import pandas as pd
from google.cloud import storage
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("CLIENT_SECRETS")
storage_client = storage.Client()

bucket_name = os.getenv("BUCKET_NAME")
path_download = "./data/raw/playstore"

In [16]:
blobs = storage_client.list_blobs(bucket_name)

for blob in blobs:
    if "community" in blob.name:
        file_name = os.path.join(path_download, blob.name.split("/")[-1])

        blob.download_to_filename(file_name)

        pd.read_csv(file_name, encoding="utf-16").to_csv(
            file_name, index=False, encoding="utf-8"
        )

In [ ]:
import pandas as pd

df_sample = pd.read_csv(
    "/Users/user/projects/scratchs/data-processing/2024/202402/20240221/data/raw/playstore/installs_io.nfttown.community_202402_overview.csv",
    encoding="utf-16",
)
df_sample

## AppStore

In [ ]:
import time
import os
import io
from datetime import datetime, timedelta

import jwt
import httpx
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

KEY_ID = os.getenv("APPLE_KEY_ID")
ISSUER_ID = os.getenv("APPLE_ISSUER_ID")
PATH_TO_KEY = os.getenv("APPLE_PRIVATE_KEY")

IAT = int(round(time.time()))
EXP = int(round(time.time() + (10 * 60.0)))  # 10 minutes timestamp

with open(PATH_TO_KEY, "r") as f:
    PRIVATE_KEY = f.read()

headers = {"alg": "ES256", "kid": KEY_ID, "typ": "JWT"}

payload = {
    "iss": ISSUER_ID,
    "iat": IAT,
    "exp": EXP,
    "aud": "appstoreconnect-v1",
}

# Create the JWT
token = jwt.encode(payload, PRIVATE_KEY, algorithm="ES256", headers=headers)

# API Request
JWT = f"Bearer {token}"
HEAD = {"Authorization": JWT}

In [ ]:
URL = "https://api.appstoreconnect.apple.com/v1/salesReports"

for day in range(1, 30):
    date = (datetime.now() - timedelta(days=day)).strftime("%Y-%m-%d")

    response = httpx.get(
        URL,
        params={
            "filter[frequency]": "DAILY",
            "filter[reportType]": "SALES",
            "filter[reportSubType]": "SUMMARY",
            "filter[vendorNumber]": os.getenv("APPLE_VENDOR_NUMBER"),
            "filter[reportDate]": date,
        },
        headers=HEAD,
    )

    if response.status_code == 200:
        df = pd.read_csv(io.BytesIO(response.content), compression="gzip", sep="\t")

        df = df.loc[df["Title"] == "ENTA"]

        if not df.empty:
            df.to_csv(
                f"./data/raw/appstore/salesReports_DAILY-SALES-SUMMARY_{date}.csv",
                index=False,
            )
    else:
        print(response.json())

In [ ]:
URL = "https://api.appstoreconnect.apple.com/v1/salesReports"

for day in range(1, 7):
    date = (datetime.now() - timedelta(days=day)).strftime("%Y-%m-%d")

    response = httpx.get(
        URL,
        params={
            "filter[frequency]": "DAILY",
            "filter[reportType]": "SALES",
            "filter[reportSubType]": "SUMMARY",
            "filter[vendorNumber]": os.getenv("APPLE_VENDOR_NUMBER"),
            "filter[reportDate]": date,
        },
        headers=HEAD,
    )

    if response.status_code == 200:
        df = pd.read_csv(io.BytesIO(response.content), compression="gzip", sep="\t")

        df = df.loc[df["Title"] == "ENTA"]

        if not df.empty:
            df.to_csv(
                f"./data/raw/appstore/salesReports_DAILY-SALES-SUMMARY_{date}.csv",
                index=False,
            )
    else:
        print(response.json())